# Creating House of PlantAgent

## Housekeeping Imports

In [1]:
import os
from dataclasses import dataclass
from langgraph.runtime import get_runtime
from langgraph.config import get_stream_writer
from langchain.agents import create_agent
import json

# For web-searching tool
from langchain.tools import tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper


## Loading API KEY

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

## System Prompt

In [3]:
SYSTEM_PROMPT = """You are an expert vegan/plant-based nutritionist and meal generator.
Your name is House of PlantAgent.

Rules:
- Think step-by-step.
- Provide a list of ingredients, quantities, and cooking instructions in return to the user query for recipe.
- If the user asks non-recipe related questions, state that your purpose is only to generate user recipes.

You have access to one tool, which you MUST use before generating each new recipe-generated response:

- search_plantbased_recipe: use this to search vegan recipe websites based on the user's stated preferences.
"""

## Defining Runtime Context

In [4]:
@dataclass # What does all this do again?
class RuntimeContext:
    """Custom runtime context schema."""
    pass

In [5]:
# can I try a simpler tool approach?
@tool
def search_vegan_recipe(query:str) -> str:
    """
    Safe wrapper that always returns a string.
    Use this as the tool to search recipes.

    Argument 'query': The search query string"""
    try:
        writer = get_stream_writer()
    except KeyError:
        def writer(msg:str):
            return msg
    wrapper = DuckDuckGoSearchAPIWrapper()
    try:
        writer("Beginning search...")
        try:
            writer("Results: ")
            result = wrapper.run(query)
            return result

        except Exception as e:
            return writer(f"Error initiating wrappter: {e}")    
    
    except Exception as e:
        return writer(f"Error initiating stream writer: {e}")


## Defining Tool

In [6]:
@tool
def search_plantbased_recipe(query:str) -> str:
    """
    Safe wrapper that always returns a string.
    Use this as the tool passed to the agent.

    Argument 'query': The search query string"""
    runtime = get_runtime(RuntimeContext)
    try:
        wrapper = DuckDuckGoSearchAPIWrapper()
        try:
            # prefer .run()
            result = wrapper.run(query)
        except Exception:
            # fallback if the wrapper method differs
            result = wrapper.run(query)

        # If result is already a string, return it.
        if isinstance(result, str):
            return result
        # If it's a dict/list/obj, try to serialise to json as safe fallback
        try:
            return json.dumps(result, default=str, ensure_ascii=False)
        except Exception:
            return str(result)
    except Exception as e:
        # Always return a string without raising so the agent gets a tool response
        return f"ERROR in search_plantbased_recipe: {type(e).__name__}: {e}"




### Testing Tool

In [7]:
# out = search_plantbased_recipe.run("tofu and noodles recipe")
# print(out[:200])



## Loading Model

In [8]:
agent = create_agent(
    model = "openai:gpt-4o",
    tools=[search_vegan_recipe],
    system_prompt = SYSTEM_PROMPT,
    context_schema = RuntimeContext(),
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
question = "Generate a recipe for tofu at dinner"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Generate a recipe for tofu at dinner
================================== Ai Message ==================================
Tool Calls:
  search_plantbased_recipe (call_TDQ1KwzbVEv4Hxi6RcFZ21Pv)
 Call ID: call_TDQ1KwzbVEv4Hxi6RcFZ21Pv
  Args:
    query: tofu dinner
================================= Tool Message =================================
Name: search_plantbased_recipe

null
================================== Ai Message ==================================

Here's a delicious and simple vegan recipe using tofu for dinner: 

### Tofu Stir Fry with Vegetables

#### Ingredients:
- **14 oz (400g) firm tofu**, drained and pressed
- **2 tablespoons soy sauce**
- **1 tablespoon sesame oil**
- **1 tablespoon vegetable oil**
- **2 cloves garlic**, minced
- **1 inch (2.5cm) piece ginger**, grated
- **1 red bell pepper**, sliced
- **1 yellow bell pepper**, sliced
- **1 cup broccoli florets**
- **1 cup sugar snap peas*

In [ ]:
# TODO_1 - replace DuckDuck with a better web search tool
# TODO_2 - return not just text but also URL up to the agent, so the agent can enhance the request.